## Manim Examples

Collection of examples using Manim

In [2]:
from manim import *
import numpy as np

config.media_width = "75%"
config.verbosity = "WARNING"

In [ ]:
%%manim -qm ShapeAndTextScene

class ShapeAndTextScene(Scene):
    def construct(self):
        